In [1]:
from datetime import datetime
import os
import warnings

import heliopy.data.omni as omni
from keras import backend as K
from matplotlib import pyplot as plt
import optuna
from optuna import visualization as viz
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy.stats import ks_2samp
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow import keras

from typing import *

warnings.filterwarnings("ignore")
plt.style.use("seaborn")

Using TensorFlow backend.


### Data Preparation

In [2]:
START_TIME_CYCLE_21 = datetime(1976, 3, 1)  
START_TIME_CYCLE_22 = datetime(1986, 9, 1)  
START_TIME_CYCLE_23 = datetime(1996, 8, 1)  
START_TIME_CYCLE_24 = datetime(2008, 12, 1)  
START_TIME_CYCLE_25 = datetime(2019, 12, 1)

INPUT_LENGTH = 100
OUTPUT_LENGTH = 24
PERCENTILE = 10
VARS_TO_PREDICT = ["N", "ABS_B", "V", "HMF_INC"]

In [3]:
def get_omni_rtn_data(start_time, end_time):
    identifier = 'OMNI_COHO1HR_MERGED_MAG_PLASMA'  # COHO 1HR data
    omni_data = omni._omni(start_time, end_time, identifier=identifier, intervals='yearly', warn_missing_units=False)
    return omni_data

In [8]:
def calc_hmf(data):
    data['HMF_INC'] = np.arctan2(-data['BT'].values, data['BN'].values)

In [11]:
cycle_21 = get_omni_rtn_data(START_TIME_CYCLE_21, START_TIME_CYCLE_22).to_dataframe()
cycle_22 = get_omni_rtn_data(START_TIME_CYCLE_22, START_TIME_CYCLE_23).to_dataframe()
cycle_23 = get_omni_rtn_data(START_TIME_CYCLE_23, START_TIME_CYCLE_24).to_dataframe()
cycle_24 = get_omni_rtn_data(START_TIME_CYCLE_24, START_TIME_CYCLE_25).to_dataframe()

calc_hmf(cycle_21)
calc_hmf(cycle_22)
calc_hmf(cycle_23)
calc_hmf(cycle_24)

In [14]:
mag_field_strength_21 = np.array(cycle_21[VARS_TO_PREDICT])
mag_field_strength_22 = np.array(cycle_22[VARS_TO_PREDICT])
mag_field_strength_23 = np.array(cycle_23[VARS_TO_PREDICT])
mag_field_strength_24 = np.array(cycle_24[VARS_TO_PREDICT])

In [18]:
def lstm_prepare_nd(multi_array, input_length=INPUT_LENGTH, output_length=OUTPUT_LENGTH):
    inputs = np.array([multi_array[i:i + input_length] 
                           for i in range(len(multi_array) - input_length - output_length + 1)])
    outputs = np.array([multi_array[i + input_length:i + input_length + output_length] 
                            for i in range(len(multi_array) - input_length - output_length + 1)])

    nan_check = np.array([multi_array[i:i + input_length + output_length] 
                              for i in range(len(multi_array) - input_length - output_length + 1)])

    inputs = inputs[np.where([~np.any(np.isnan(i)) for i in nan_check])]
    outputs = outputs[np.where([~np.any(np.isnan(i)) for i in nan_check])]

    print("Input shape:", inputs.shape)
    print("Output shape:", outputs.shape)
    print("Any Nans?:", np.any(np.isnan(outputs)) or np.any(np.isnan(inputs)))
    return inputs, outputs

### Train/Val/Test

In [19]:
inputs_21, outputs_21 = lstm_prepare_nd(mag_field_strength_21)
inputs_23, outputs_23 = lstm_prepare_nd(mag_field_strength_23)
inputs_val, outputs_val = lstm_prepare_nd(mag_field_strength_22)
inputs_test, outputs_test = lstm_prepare_nd(mag_field_strength_24)

Input shape: (2524, 100, 4)
Output shape: (2524, 24, 4)
Any Nans?: False
Input shape: (83261, 100, 4)
Output shape: (83261, 24, 4)
Any Nans?: False
Input shape: (10436, 100, 4)
Output shape: (10436, 24, 4)
Any Nans?: False
Input shape: (87202, 100, 4)
Output shape: (87202, 24, 4)
Any Nans?: False


In [20]:
inputs_train = np.concatenate([inputs_21, inputs_23])
outputs_train = np.concatenate([outputs_21, outputs_23])

### Normalise

In [22]:
training_field_strength = np.concatenate((mag_field_strength_21, mag_field_strength_23))

In [32]:
inputs_train_norm = (inputs_train - np.nanmean(training_field_strength, axis=0)) / np.nanstd(training_field_strength, axis=0)
inputs_val_norm = (inputs_val - np.nanmean(training_field_strength, axis=0)) / np.nanstd(training_field_strength, axis=0)
inputs_test_norm = (inputs_test - np.nanmean(training_field_strength, axis=0)) / np.nanstd(training_field_strength, axis=0)

### LSTM

In [45]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_initial (LSTM)          (None, None, 32)          4736      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 4)           132       
Total params: 4,868
Trainable params: 4,868
Non-trainable params: 0
_________________________________________________________________


In [43]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(32, name="lstm_initial", input_shape=(None, 4), return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(4, name="dense_final", activation="linear")),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train_norm[:, -24:], outputs_train, validation_data=(inputs_val_norm[:, -24:], outputs_val),
          batch_size=32, epochs=500, verbose=2,
          callbacks=[keras.callbacks.EarlyStopping(restore_best_weights=True, patience=10)])


Epoch 1/500
2681/2681 - 16s - loss: 42920.1367 - mae: 101.6629 - val_loss: 29044.2402 - val_mae: 83.5657
Epoch 2/500
2681/2681 - 14s - loss: 27819.7617 - mae: 80.4650 - val_loss: 17413.1328 - val_mae: 63.2139
Epoch 3/500
2681/2681 - 15s - loss: 16906.2129 - mae: 60.6257 - val_loss: 9427.5967 - val_mae: 43.8149
Epoch 4/500
2681/2681 - 14s - loss: 9446.4443 - mae: 41.9480 - val_loss: 4660.6938 - val_mae: 26.8412
Epoch 5/500
2681/2681 - 14s - loss: 5088.0586 - mae: 28.2903 - val_loss: 2673.0623 - val_mae: 20.7352
Epoch 6/500
2681/2681 - 14s - loss: 3313.8479 - mae: 23.6302 - val_loss: 2545.7776 - val_mae: 22.7200
Epoch 7/500
2681/2681 - 15s - loss: 2904.8716 - mae: 22.9693 - val_loss: 1496.6523 - val_mae: 15.8366
Epoch 8/500
2681/2681 - 15s - loss: 1687.5514 - mae: 16.1638 - val_loss: 1077.0825 - val_mae: 13.9952
Epoch 9/500
2681/2681 - 15s - loss: 1347.5936 - mae: 14.7806 - val_loss: 961.5799 - val_mae: 13.4686
Epoch 10/500
2681/2681 - 14s - loss: 1237.2777 - mae: 14.2998 - val_loss: 899

In [44]:
model.evaluate(inputs_test_norm[:, -24:], outputs_test, verbose=2)

2726/2726 - 4s - loss: 862.2397 - mae: 12.2619


[862.2396850585938, 12.261914253234863]

### Enc-Dec

In [53]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(32, name="lstm_initial", input_shape=(None, 4)),
        keras.layers.RepeatVector(24),
        keras.layers.LSTM(32, name="lstm_second", return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(4, name="dense_final", activation="linear")),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train_norm[:, -24:], outputs_train, validation_data=(inputs_val_norm[:, -24:], outputs_val),
          batch_size=32, epochs=500, verbose=2,
          callbacks=[keras.callbacks.EarlyStopping(restore_best_weights=True, patience=10)])


Epoch 1/500
2681/2681 - 32s - loss: 43256.1328 - mae: 102.0516 - val_loss: 29504.4727 - val_mae: 84.2721
Epoch 2/500
2681/2681 - 32s - loss: 28397.6562 - mae: 81.3791 - val_loss: 17916.0234 - val_mae: 64.2242
Epoch 3/500
2681/2681 - 31s - loss: 17403.4199 - mae: 61.6775 - val_loss: 9786.0732 - val_mae: 44.8820
Epoch 4/500
2681/2681 - 30s - loss: 9779.6396 - mae: 42.9435 - val_loss: 4847.8008 - val_mae: 27.5768
Epoch 5/500
2681/2681 - 30s - loss: 5263.9385 - mae: 28.8244 - val_loss: 2725.8521 - val_mae: 20.7626
Epoch 6/500
2681/2681 - 30s - loss: 3364.1909 - mae: 23.7033 - val_loss: 2531.9863 - val_mae: 22.5718
Epoch 7/500
2681/2681 - 31s - loss: 3013.7195 - mae: 23.6316 - val_loss: 2687.5671 - val_mae: 23.8510
Epoch 8/500
2681/2681 - 30s - loss: 3004.7771 - mae: 23.7871 - val_loss: 2697.4863 - val_mae: 23.9205
Epoch 9/500
2681/2681 - 28s - loss: 3004.6501 - mae: 23.7911 - val_loss: 2696.4646 - val_mae: 23.9086
Epoch 10/500
2681/2681 - 30s - loss: 3004.5981 - mae: 23.7927 - val_loss: 26

KeyboardInterrupt: 

### Attention

In [ ]:
# https://stackoverflow.com/questions/62948332/how-to-add-attention-layer-to-a-bi-lstm/62949137#62949137

In [67]:
from tensorflow.keras import backend as K
class CustomAttention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(CustomAttention, self).__init__()
        
    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                               initializer="zeros")
        super(CustomAttention, self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [106]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(32, name="lstm_initial", input_shape=(24, 4), return_sequences=True),
        CustomAttention(return_sequences=True),
        keras.layers.LSTM(32, name="lstm_second", return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(4, name="dense_final", activation="linear")),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train_norm[:, -24:], outputs_train, validation_data=(inputs_val_norm[:, -24:], outputs_val),
          batch_size=32, epochs=500, verbose=2,
          callbacks=[keras.callbacks.EarlyStopping(restore_best_weights=True, patience=10)])


Epoch 1/500
2681/2681 - 35s - loss: 43010.5430 - mae: 101.7500 - val_loss: 29149.6367 - val_mae: 83.7246
Epoch 2/500
2681/2681 - 31s - loss: 28042.7207 - mae: 80.8158 - val_loss: 17638.2891 - val_mae: 63.6769
Epoch 3/500
2681/2681 - 34s - loss: 17134.1328 - mae: 61.1237 - val_loss: 9595.6865 - val_mae: 44.3188
Epoch 4/500
2681/2681 - 32s - loss: 9600.2656 - mae: 42.4284 - val_loss: 4746.8755 - val_mae: 27.1750
Epoch 5/500
2681/2681 - 34s - loss: 5169.5757 - mae: 28.5507 - val_loss: 2696.8542 - val_mae: 20.7153
Epoch 6/500
2681/2681 - 33s - loss: 3334.8081 - mae: 23.6412 - val_loss: 2540.6685 - val_mae: 22.6493
Epoch 7/500
2681/2681 - 31s - loss: 2305.2158 - mae: 19.0693 - val_loss: 1009.8711 - val_mae: 12.4129
Epoch 8/500
2681/2681 - 33s - loss: 1228.6946 - mae: 13.5021 - val_loss: 722.8624 - val_mae: 11.3882
Epoch 9/500
2681/2681 - 30s - loss: 936.4607 - mae: 12.0814 - val_loss: 615.4857 - val_mae: 10.6101
Epoch 10/500
2681/2681 - 32s - loss: 802.1810 - mae: 11.2938 - val_loss: 561.20

In [107]:
model.evaluate(inputs_test_norm[:, -24:], outputs_test, verbose=2)

2726/2726 - 7s - loss: 500.0357 - mae: 9.2908


[500.0356750488281, 9.290789604187012]

In [130]:
model.evaluate(inputs_train_norm[:, -24:], outputs_train, verbose=2)

2681/2681 - 7s - loss: 668.3700 - mae: 10.2056


[668.3699951171875, 10.20560359954834]

In [131]:
model.evaluate(inputs_val_norm[:, -24:], outputs_val, verbose=2)

327/327 - 1s - loss: 485.4785 - mae: 9.3864


[485.47845458984375, 9.38636589050293]

### More complex attention

In [146]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(64, name="lstm_initial", input_shape=(24, 4), return_sequences=True),
        keras.layers.LSTM(128, return_sequences=True),
        CustomAttention(return_sequences=True),
        keras.layers.LSTM(128, name="lstm_second", return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(4, name="dense_final", activation="linear")),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train_norm[:, -24:], outputs_train, validation_data=(inputs_val_norm[:, -24:], outputs_val),
          batch_size=32, epochs=500, verbose=2,
          callbacks=[keras.callbacks.EarlyStopping(restore_best_weights=True, patience=10)])


Epoch 1/500
2681/2681 - 91s - loss: 24372.3184 - mae: 72.0005 - val_loss: 6000.4502 - val_mae: 32.0991
Epoch 2/500


KeyboardInterrupt: 

### Attention with larger input

In [ ]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(32, name="lstm_initial", input_shape=(100, 4), return_sequences=True),
        CustomAttention(return_sequences=True),
        keras.layers.LSTM(32, name="lstm_second", return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(4, name="dense_final", activation="linear")),
        keras.layers.Cropping1D(cropping=(0, 76))
    ]
)
model.summary()

optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train_norm[:, -100:], outputs_train, validation_data=(inputs_val_norm[:, -100:], outputs_val),
          batch_size=32, epochs=500, verbose=2,
          callbacks=[keras.callbacks.EarlyStopping(restore_best_weights=True, patience=10)])


Model: "sequential_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_initial (LSTM)          (None, 100, 32)           4736      
_________________________________________________________________
custom_attention_43 (CustomA (None, 100, 32)           132       
_________________________________________________________________
lstm_second (LSTM)           (None, 100, 32)           8320      
_________________________________________________________________
time_distributed_37 (TimeDis (None, 100, 4)            132       
_________________________________________________________________
cropping1d_12 (Cropping1D)   (None, 24, 4)             0         
Total params: 13,320
Trainable params: 13,320
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500


### Baselines

In [129]:
# Last 24
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = inp[:, -24:, :]
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

1627.9448
16.589468

1189.4584
14.248516

1226.7482
14.479452



In [128]:
# Last 1, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(inp[:, -1:, :], 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

797.5559
10.708757

584.09076
9.458246

596.2808
9.441223



In [137]:
# Mean of last 24, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(np.reshape(np.mean(inp[:, -24:, :], axis=1), (len(inp), 1, 4)), 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

1372.0238
14.951957

1006.5172
12.818467

1035.1685
13.03555



In [138]:
# Mean of last 100, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(np.reshape(np.mean(inp[:, -100:, :], axis=1), (len(inp), 1, 4)), 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

2501.9717
21.087082

2002.0676
18.584085

1891.9309
18.228844



In [139]:
# Median of last 24, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(np.reshape(np.median(inp[:, -24:, :], axis=1), (len(inp), 1, 4)), 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

1466.1205
15.213919

1083.7523
13.031278

1107.4498
13.255629



In [140]:
# Max of last 24, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(np.reshape(np.max(inp[:, -24:, :], axis=1), (len(inp), 1, 4)), 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

2003.5787
20.203812

1361.8109
17.362165

1430.5306
17.416336



In [141]:
# Min of last 24, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(np.reshape(np.min(inp[:, -24:, :], axis=1), (len(inp), 1, 4)), 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

1933.7264
16.220797

1456.5176
14.380861

1506.6075
14.450279



In [144]:
# 24 ago, repeated
for inp, out in zip([inputs_train, inputs_val, inputs_test], [outputs_train, outputs_val, outputs_test]):
    outputs_pred = np.repeat(inp[:, -24:-23, :], 24, axis=1)
    print(np.mean((out - outputs_pred) ** 2))
    print(np.mean(np.abs((out - outputs_pred))))
    print("")

2443.1892
20.533897

1775.3417
17.520683

1833.1857
17.77644

